# Maximum Likelyhood Estimation (MLE) and Maximum A Posteriori (MAP) estimation

## Warmup to MLE and MAP

We will introduce the ideas of MLE and MAP in a simple discrete case.

#### MLE

You are given a coin and you are told that it is weighted to come up heads with probability $\theta$.  You toss the coin $3$ times and obtain the sequence HHT.  Based only on this information, what is your best estimate for $\theta$?

One way to address this rigorously is to rephrase the question this way:  of all coins with weightings $\theta \in [0,1]$, which value of $p$ gives the **maximum likelyhood** of producing the sequence HHT?  First make a guess about what is reasonable!

The likelyhood of a coin weighted $\theta$ producing HHT is 

$$L(\theta) = \theta^2(1-\theta)$$

The maximum value of this function over $[0,1]$ can be found using differential calculus:

$$
\begin{align*}
\frac{\textrm{d} L}{\textrm{d} \theta} 
&= 2\theta(1-\theta) - \theta^2\\
&=2\theta - 2\theta^2 - \theta^2\\
&=2\theta - 3\theta^2
\end{align*}
$$

So the derivative is $0$ when $\theta = 0$ or $\theta = \frac{2}{3}$.  As $L(0) = L(1) = 0$ and $L(\frac{2}{3}) > 0$ we have that the global maximum of $L$ occurs at $\theta = \frac{2}{3}$.  

Does this agree with the guess you made before doing the Calculus?

Let's do the same thing, but imagine that you tossed the coin $n$ times, got $m$ heads and $n-m$ tails.

The probability of a coin weighted $\theta$ producing $m$ heads and $n-m$ tails.

$$L(\theta) = \theta^n(1-\theta)^{n-m}$$

We could differentiate this directly, but we will get a cleaner result if we use logarithmic differentiation.

$$
\begin{align*}
\log(L)(\theta) &= \log(\theta^m(1-\theta)^{n-m})\\
&= m\log(\theta) + (n-m)\log(1-\theta)\\
\frac{\textrm{d}}{\textrm{d}\theta} \log(L(\theta)) = \frac{m}{\theta} - \frac{n-m}{1-\theta}
\end{align*}
$$

Setting this equal to zero we have

$$
\begin{align*}
\frac{m}{\theta} - \frac{n-m}{1-\theta} &= 0\\
m(1-\theta) - (n-m)\theta &= 0\\
m - m\theta - n\theta + m\theta &= 0\\
\theta &= \frac{m}{n} 
\end{align*}
$$

which should also agree with your intuition!  

#### MAP

Real world coins have $\theta \approx \frac{1}{2}$.  In the situation above we are **told** that the coin (might be) biased, and so it is reasonable to guess that $\theta = \frac{4}{5}$ if you observe $4$ heads and $1$ tail.  Without being given any such information, we might instead think that the distribution of weights could look something like this:

<p align="center">

<img src="math_hour_1_assets/beta_prior.png" alt="beta prior" style="width:200px;"/>

</p>

This is an example of a "prior distribution" on the parameter $\theta$.  This particular prior is 

$$
k \theta^8 (1-\theta)^8
$$

where the constant $k$ has been chosen to make the integral equal to $1$ over $[0,1]$ (it turns out that $k = \frac{1}{B(9,9)}$ where $B$ is the "beta" function. See [beta distribution](https://en.wikipedia.org/wiki/Beta_distribution) for more info).

Assume that we flip our coin $5$ times and obtain $4$ heads as before. Instead of estimating $\theta = \frac{m}{n}$ we should now guess something in between $\frac{4}{5}$ and $\frac{1}{2}$.  How can we calculate it exactly?  Bayes' Rule to the rescue!

$$
\begin{align*}
P(\theta | \textrm{Data}) 
&\propto P(\textrm{Data}| \theta) \cdot P(\theta)\\
&\propto \theta^4(1-\theta)^1 \cdot \theta^8(1-\theta)^8\\
&\propto \theta^{4+8}(1-\theta)^{1+8}\\
&= \theta^{12} (1 - \theta)^9
\end{align*}
$$

We have maximized this kind of thing before:  the maximum value occurs at $\hat{\theta} = \frac{12}{21}$.

MAP (in this case) was equivalent to MLE with 8 extra heads and tails than what we actually observed.  This is a nice interpretation:  it is one way of making the statement that "most coins have $\theta \approx \frac{1}{2}$" precise.

In general MLE is equivalent to MAP with a uniform prior.

## MLE estimates of mean and variance

We now leave the discrete world.

Assume that $n$ samples $x_1, x_2, x_3, \dots , x_n$ are drawn from a normal distribution with (unknown) mean $\mu$ and (unknown) variance $\sigma^2$.

We want to find the maximum likelyhood estimates $\hat{\mu}$ and $\hat{\sigma}^2$ of these parameters.  You might be able to guess what these are!  Absent any other informationl, what would your best guess for $\mu$ and $\sigma$ be?

In other words, we want to maximize

$$
\operatorname{L}(\mu, \sigma) = \prod_{i=1}^{n} \frac{1}{\sqrt{2\pi} \sigma} \exp\left(-\frac{(x_i-\mu)^2}{2\sigma^2}\right)
$$

Again, it is more convenient to turn this product into a sum using the logarithm. This is the **log likelyhood**.

$$
\operatorname{LL}(\mu, \sigma) = \log\left( \frac{1}{\sqrt{2\pi} \sigma} \exp\left(-\frac{(x_i-\mu)^2}{2\sigma^2}\right) \right)
$$

After a little algebra we have 

$$
\operatorname{LL}(\mu, \sigma)  = \left[ -n \log{\sqrt{2\pi}} - n\log(\sigma) - \sum_{i=1}^{n} \frac{(x_i-\mu)^2}{2\sigma^2} \right]
$$

This is a lot of minus signs, so we can instead think about minimizing the **negative log likelyhood**

$$
\operatorname{NLL}(\mu, \sigma) = \left[ n \log{\sqrt{2\pi}} + n\log(\sigma) + \sum_{i=1}^{n} \frac{(x_i-\mu)^2}{2\sigma^2}\right]
$$

I will switch to using $\ell$ in place of $\operatorname{NLL}$ to make it look pretty.

We can now take partial derivatives

$$
\begin{align*}
\frac{\partial \ell}{\partial \mu} &= \sum_{i=1}^{n} \frac{(x_i-\mu)}{\sigma^2}\\
\frac{\partial \ell}{\partial \mu} &= \frac{n}{\sigma} - \sum_{i=1}^{n} \frac{(x_i-\mu)^2}{\sigma^3}
\end{align*}
$$

Setting both equal to $0$ and solving we obtain

$$
\begin{align*}
\hat{\mu} &= \frac{1}{n} \sum_{i=1}^{n} x_i = \bar{x}\\
\hat{\sigma}^2 &= \frac{1}{n} \sum_{i=1}^{n} (x_i - \bar{x})^2
\end{align*}
$$


## Where to go from here?

We will use MLE constantly in math hour!  In particular, MLE comes up in:

* Linear Regression
* Logistic Regression
* Gaussian Discriminant Analysis

More generally, MLE follows a common machine learning pattern which we will see again and again:

* We have some data.
* We create a model for a data generating process which depends on some parameters.
* We specify a "loss function": how do we judge "how well" a given set of parameters models the data?
* We minimize the loss function, often by using gradient descent when exact solutions are unavailable.